# Purpose: To run the full segmentation using the best scored method from 2_compare_auto_to_manual_threshold

Date Created: January 7, 2022

Dates Edited: January 26, 2022 - changed the ogd severity study to be the otsu data as the yen data did not run on all samples.

*Step 1: Import Necessary Packages*

In [4]:
# import major packages
import numpy as np
import matplotlib.pyplot as plt
import skimage
import PIL as Image
import os
import pandas as pd

# import specific package functions
from skimage.filters import threshold_otsu
from skimage import morphology
from scipy import ndimage
from skimage.measure import label
from skimage import io
from skimage import measure

__LPS-MSG-SOD_Data__

In [5]:
im_folder_location = '/Users/hhelmbre/Desktop/microfiber/ricks_iba1_images/LPS_high_mag_to_segment/60x'

In [6]:
im_paths = []
files = []
for file in os.listdir(im_folder_location):
    if file.endswith(".tif"):
        file_name = os.path.join(im_folder_location, file)
        files.append(file)
        im_paths.append(file_name)

In [7]:
files

['2019-02-25_NT_serum_24h_60x_scan_1.tif',
 '2019-02-25_NT_serum_24h_60x_scan_2.tif',
 '2019-02-25_NT_serum_24h_60x_scan_3.tif',
 '2019-02-18_DAPI_Iba_LPS_100ng_0-24_24h_60x_scan_2.tif',
 '2019-02-18_DAPI_Iba_LPS_1ug_0-24_24h_60x_scan_2.tif',
 '2019-02-18_DAPI_Iba_LPS_1ug_0-24_24h_60x_scan_3.tif',
 '2019-02-18_DAPI_Iba_LPS_100ng_0-24_24h_60x_scan_3.tif',
 '2019-02-18_DAPI_Iba_LPS_100ng_0-24_24h_60x_scan_1.tif',
 '2019-02-18_DAPI_Iba_LPS_1ug_0-24_24h_60x_scan_1.tif',
 '2019-02-18_DAPI_Iba_NT_72h_60x_scan_3.tif',
 '2019-02-25_LPS_1ug_serum_24h_60x_scan_1.tif',
 '2019-02-18_DAPI_Iba_LPS_100ng_0-72_72h_60x_scan_1.tif',
 '2019-02-18_DAPI_Iba_LPS_1ug_0-72_72h_60x_scan_1.tif',
 '2019-02-18_DAPI_Iba_NT_72h_60x_scan_2.tif',
 '2019-02-25_LPS_1ug_serum_24h_60x_scan_2.tif',
 '2019-02-18_DAPI_Iba_LPS_100ng_0-72_72h_60x_scan_2.tif',
 '2019-02-18_DAPI_Iba_LPS_1ug_0-72_72h_60x_scan_2.tif',
 '2019-02-25_LPS_1ug_serum_24h_60x_scan_3.tif',
 '2019-02-18_DAPI_Iba_NT_72h_60x_scan_1.tif',
 '2019-02-18_DAPI_I

In [8]:
properties_list = ('area', 'bbox_area', 'centroid', 'convex_area', 
                   'eccentricity', 'equivalent_diameter', 'euler_number', 
                   'extent', 'filled_area', 'major_axis_length', 
                   'minor_axis_length', 'orientation', 'perimeter', 'solidity')

In [67]:
source_dir = '/Users/hhelmbre/Desktop/microfiber/ricks_iba1_images/LPS_high_mag_to_segment/segmented/'

To determine the size of the small object we should remove we took the size of microglia which are approximately 1600 um^2 according to (https://jneuroinflammation.biomedcentral.com/articles/10.1186/s12974-014-0182-7)

The confocal settings for the LPS study at 60x provided a 1 pixel = 2.4136 um conversion. To ensure we didn't cut off any potential microglia, we chose a cutoff on the small end to be 1/2 of the average which converts to 138 pixels ^2 used in the min_size of the remove small objects function from sci-kit image.

In [69]:
j = 0
for image in im_paths:
    
    short_im_name = image.rsplit('/', 1)
    short_im_name = short_im_name[1]
    
    im = io.imread(image)
    IM_MAX= np.max(im, axis=0)
    microglia_im = IM_MAX[1,:,:]
    #otsu threshold
    thresh_otsu = skimage.filters.threshold_otsu(microglia_im)
    binary_otsu = microglia_im > thresh_otsu
    new_binary_otsu = morphology.remove_small_objects(binary_otsu, min_size=138)
    new_binary_otsu = ndimage.binary_fill_holes(new_binary_otsu)
    label_image = measure.label(new_binary_otsu)
    props = measure.regionprops_table(label_image, properties=(properties_list))
    
    np.save(str(source_dir + short_im_name[:-4] + '_otsu_thresh'), new_binary_otsu)
    plt.imsave(str(source_dir + short_im_name[:-4] + '_otsu_thresh.png'), label_image)
    
    
    if j == 0:
        df = pd.DataFrame(props)
        df['filename'] = image
    else:
        df2 = pd.DataFrame(props)
        df2['filename'] = image
        df = df.append(df2)

    j = 1
    
    

In [53]:
df['circularity'] = 4*np.pi*df.area/df.perimeter**2

In [54]:
df['aspect_ratio'] = df.major_axis_length/df.minor_axis_length

In [55]:
df

,area,bbox_area,centroid-0,centroid-1,convex_area,eccentricity,equivalent_diameter,euler_number,extent,filled_area,major_axis_length,minor_axis_length,orientation,perimeter,solidity,filename,circularity,aspect_ratio
0,101,143,5.237624,254.653465,105,0.700071,11.340070,1,0.706294,101,13.531434,9.662432,-0.539565,37.727922,0.961905,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.891673,1.400417
1,92,171,3.608696,835.097826,137,0.864835,10.823033,1,0.538012,92,20.242625,10.162937,1.495310,64.698485,0.671533,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.276191,1.991809
2,240,320,6.962500,957.841667,252,0.616664,17.480775,1,0.750000,240,19.847430,15.624423,1.501335,58.284271,0.952381,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.887807,1.270282
3,87,162,3.275862,1405.540230,117,0.851214,10.524820,1,0.537037,87,17.847114,9.366498,-1.489181,58.319805,0.743590,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.321438,1.905420
4,183,285,11.464481,1366.803279,209,0.569676,15.264430,1,0.642105,183,17.177540,14.117689,-0.715505,53.455844,0.875598,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.804768,1.216739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,153,238,1458.843137,963.751634,190,0.688849,13.957279,1,0.642857,153,17.254196,12.507656,-1.083417,63.491378,0.805263,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.476949,1.379491
526,153,252,1462.725490,920.431373,173,0.669893,13.957279,1,0.607143,153,16.608948,12.331439,-0.034563,53.526912,0.884393,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.671053,1.346878
527,94,192,1466.372340,1313.914894,135,0.852347,10.940042,1,0.489583,94,17.944863,9.384739,-0.462740,57.077164,0.696296,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.362588,1.912132
528,211,378,1472.725118,427.161137,268,0.425300,16.390654,1,0.558201,211,18.268863,16.534283,1.160228,71.219300,0.787313,/Users/hhelmbre/Desktop/microfiber/ricks_iba1_...,0.522753,1.104908


In [39]:
df.to_csv('/Users/hhelmbre/Desktop/microfiber/lps_otsu_features.csv' )

In [40]:
%load_ext watermark

%watermark -v -m -p numpy,pandas,scipy,skimage,matplotlib,wget

%watermark -u -n -t -z

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.8.0

numpy     : 1.21.5
pandas    : 1.3.5
scipy     : 1.3.1
skimage   : 0.17.2
matplotlib: 3.1.1
wget      : 3.2

Compiler    : Clang 4.0.1 (tags/RELEASE_401/final)
OS          : Darwin
Release     : 20.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit

Last updated: Wed May 18 2022 14:00:31PDT

